In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [2]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('Dataset/intents.json').read()
intents = json.loads(data_file)

## <b><i> <-- Pre-Processing Data --> </i></b>

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        # add documents in the corpus
        documents.append((w, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#### <i> we will lemmatize each word and remove duplicate words from the list. Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.</i>

In [9]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print(len(documents), "documents")

# classes = intents
print(len(classes), "classes", classes)

#words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


#### <i> we will create the training data in which we will provide the input and the output. Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.</i>

In [10]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word -create base word, in attemt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    # output is a '0' for each tag and '1' for current tag(for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
# shuffle our feature and turn into np. array
random. shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents

train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

Training data created


<ipython-input-10-3669c643632c>:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


## <b><i> <-- Build a Model --> </i></b>
<i>now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved 100% accuracy on our model.</i>

In [11]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

# compile model. Stochastic gradient descent with  Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)

model.save('chatbot_model.h5', hist)

print("Model Created")

Epoch 1/200
10/10 [==============================] - 0s 901us/step - loss: 2.2554 - accuracy: 0.1064
Epoch 2/200
10/10 [==============================] - 0s 695us/step - loss: 2.0946 - accuracy: 0.2340
Epoch 3/200
10/10 [==============================] - 0s 898us/step - loss: 2.0487 - accuracy: 0.2553
Epoch 4/200
10/10 [==============================] - 0s 698us/step - loss: 1.9902 - accuracy: 0.3191
Epoch 5/200
10/10 [==============================] - 0s 801us/step - loss: 1.8733 - accuracy: 0.4468
Epoch 6/200
10/10 [==============================] - 0s 898us/step - loss: 1.6044 - accuracy: 0.6170
Epoch 7/200
10/10 [==============================] - 0s 599us/step - loss: 1.7497 - accuracy: 0.4468
Epoch 8/200
10/10 [==============================] - 0s 798us/step - loss: 1.5442 - accuracy: 0.5319
Epoch 9/200
10/10 [==============================] - 0s 596us/step - loss: 1.4444 - accuracy: 0.4468
Epoch 10/200
10/10 [==============================] - 0s 698us/step - loss: 1.1343 - accura